In [1]:
import pandas as pd
from pathlib import Path
from collections import Counter
import os
import difflib
import math

In [2]:
DATA_PATH = Path("/home/pengfei/data/pdtb_v2/data/raw/wsj/")

In [3]:
pdtb2 = pd.read_csv("../pdtb2/pdtb2.csv")

/home/pengfei/miniconda3/envs/stanfordnlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


in terms of segment number:
single, double, trible = (38584, 1948, 51)

how many distance away from double span:


    Counter(a).most_common()
    
    
    all of them is within 5

How many distances away:
   
   
   Counter({0: 37713,
         1: 2598,
         2: 193,
         6: 7,
         12: 1,
         9: 3,
         3: 50,
         5: 5,
         18: 2,
         4: 18,
         13: 1,
         8: 4,
         15: 1,
         62: 1,
         19: 1,
         10: 1,
         16: 1})

need correction distance:
    [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16, 18, 19, 62]

In [5]:
# write functions

In [13]:
def _get_files(data_path=DATA_PATH):
    """Returns (list): [('00', 'wsj_0000'), ..., ('24', 'wsj_2400']"""
    sections = os.listdir(data_path)
    ret = []
    for sec in sections:
        for filename in os.listdir(data_path/sec):
            ret.append((sec, filename))
    return ret

def get_batch(section, filenumber):
    """
    Args: 
            section(int)
            filenumber(int)
    Returns:
            (list[int]): list of index with all the file with the same format in that batch"""
    return pdtb2.index[(pdtb2['Section'] == section) & (pdtb2.FileNumber == filenumber)].tolist()


def _get_arg(char_span_list, rawtext):
    arg = ''
    for span in char_span_list:
        arg += rawtext[int(span[0])-2:int(span[1])-2]
    return arg

def _get_span_list(span):
    """
    Args:
            span(str): "34..96;97..101"
    Returns:
            ret(list[list[str]]):
            if one arg has one segment: [[34,96]]
            if one arg has two segment: example [[34, 96], [97, 101]]
    """
    if span == '':
        return []
    spans = span.split(';')
    return [[int(k) for k in o.split('..')] for o in spans ]

def check_number_of_errors():
    count = 0
    single = 0
    double = 0
    triple = 0
    files =  _get_files()
    for (section, filename) in files:
        with open(DATA_PATH/section/filename, encoding='latin-1') as f:
            rawtext = f.read()
            rawtext = rawtext.replace('\n', '')
        section = int(section)
        filenumber = int(filename[-2:])
        batch = get_batch(section, filenumber)
        for i in batch:
            # for arg1
            arg1_char_span_string = pdtb2.loc[i, 'Arg1_SpanList']
            arg1_char_span_list = _get_span_list(arg1_char_span_string)
            arg1_rawtext = pdtb2.loc[i, 'Arg1_RawText']
            if _get_arg(arg1_char_span_list, rawtext) != arg1_rawtext:
                if len(arg1_char_span_list) == 1:
                    single += 1
                elif len(arg1_char_span_list) == 2:
                    double += 1
                else:
                    triple += 1
    print("error numbers for arg1  single:", single, " double: ", double, " triple: ", triple)
    count = 0
    single = 0
    double = 0
    triple = 0
    files =  _get_files()
    for (section, filename) in files:
        with open(DATA_PATH/section/filename, encoding='latin-1') as f:
            rawtext = f.read()
            rawtext = rawtext.replace('\n', '')
        section = int(section)
        filenumber = int(filename[-2:])
        batch = get_batch(section, filenumber)
        for i in batch:
            # for arg2
            arg2_char_span_string = pdtb2.loc[i, 'Arg2_SpanList']
            arg2_char_span_list = _get_span_list(arg2_char_span_string)
            arg2_rawtext = pdtb2.loc[i, 'Arg2_RawText']
            if _get_arg(arg2_char_span_list, rawtext) != arg2_rawtext:
                if len(arg2_char_span_list) == 1:
                    single += 1
                elif len(arg2_char_span_list) == 2:
                    double += 1
                else:
                    triple += 1
    print("error numbers for arg2  single:", single, " double: ", double, " triple: ", triple)
    
def get_length(list):
    length = 0
    for o in list:
        length += o[1] - o[0]
    return length


def get_start_end(span_list, rawtext, tolerance_size):
    if span_list == []:
        return 0, len(rawtext)-1
    tolerance_size += 1
    start = span_list[0][0]
    end = span_list[len(span_list)-1][1]
    if start-tolerance_size<0:
        start = 0
    else:
        start -= tolerance_size
    if end + tolerance_size>len(rawtext)-1:
        end = len(rawtext) -1
    else:
        end += len(rawtext)+tolerance_size
    
    return start, end

def correct_span(span_list, arg_rawtext, rawtext):
    """
    Args: 
            span_list[list[[int, int], ...]]
            arg_rawtext(str)
            rawtext(str): doc raw text
    Returns: 
            corrected_span(str): [34, 39]
    """
    corrected_span = ''
    tolerance_size = abs(get_length(span_list) - len(arg_rawtext))
    start, end = get_start_end(span_list, rawtext, tolerance_size)

    s = difflib.SequenceMatcher(None, rawtext[start: end], arg_rawtext)
    for i, j, n in s.get_matching_blocks():
        if n!=0:
            corrected_span += f'{start+i+2}..{start+i+n+2};' # here is a offset of 2 char we need to consider
    corrected_span = corrected_span[:-1]
    return corrected_span

def sanity_check(corrected_span, rawtext, arg_rawtext):
    """Args: 
            corrected_span(str)
            rawtext(str): doc string
            arg_rawtext(str): gold argument string
    Returns: 
            (boolean): the corrected span is corrected"""
    span_list_corrected = _get_span_list(corrected_span)
    return _get_arg(span_list_corrected, rawtext) == arg_rawtext

In [14]:
def correct_docs():
    files =  _get_files()
    count = 0
    print(" --- correction start ----")
    for (section, filename) in files:
        print('section:', section, ' files: ', filename)
        with open(DATA_PATH/section/filename, encoding='latin-1') as f:
            rawtext = f.read()
        section = int(section)
        filenumber = int(filename[-2:])
        batch = get_batch(section, filenumber)
        for i in batch:
            # for arg1
            arg1_char_span_string = pdtb2.loc[i, 'Arg1_SpanList']
            arg1_char_span_list = _get_span_list(arg1_char_span_string)
            arg1_rawtext = pdtb2.loc[i, 'Arg1_RawText']
            if _get_arg(arg1_char_span_list, rawtext) != arg1_rawtext:
                corrected_span = correct_span(arg1_char_span_list, arg1_rawtext, rawtext)
                arg1_char_span_list_corrected = _get_span_list(corrected_span)

                count += 1
                check = sanity_check(corrected_span, rawtext, arg1_rawtext)
                if check == False:
                    print("Attention!")
                pdtb2.loc[i, 'Arg1_SpanList'] = corrected_span
                

            # for arg2
            arg2_char_span_string = pdtb2.loc[i, 'Arg2_SpanList']
            arg2_char_span_list = _get_span_list(arg2_char_span_string)
            arg2_rawtext = pdtb2.loc[i, 'Arg2_RawText']
            if _get_arg(arg2_char_span_list, rawtext) != arg2_rawtext:
                corrected_span = correct_span(arg2_char_span_list, arg2_rawtext, rawtext)
                check = sanity_check(corrected_span, rawtext, arg1_rawtext)
                count += 1
                check = sanity_check(corrected_span, rawtext, arg2_rawtext)
                if check == False:
                    print("Attention!")
                pdtb2.loc[i, 'Arg2_SpanList'] = corrected_span
    print("--- correction finished ------")
    return pdtb2, count

In [ ]:
pdtb2, count = correct_docs()

 --- correction start ----
section: 07  files:  wsj_0793
section: 07  files:  wsj_0739
Attention!
Attention!
Attention!
section: 07  files:  wsj_0712
section: 07  files:  wsj_0777
Attention!
section: 07  files:  wsj_0709
section: 07  files:  wsj_0797
section: 07  files:  wsj_0799
Attention!
Attention!
Attention!
Attention!
section: 07  files:  wsj_0751
section: 07  files:  wsj_0781
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
section: 07  files:  wsj_0782
section: 07  files:  wsj_0740
section: 07  files:  wsj_0732
section: 07  files:  wsj_0728
Attention!
section: 07  files:  wsj_0766
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Att

Attention!
Attention!
Attention!
section: 05  files:  wsj_0536
section: 05  files:  wsj_0580
section: 05  files:  wsj_0561
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
section: 05  files:  wsj_0568
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
section: 05  files:  wsj_0538
section: 05  files:  wsj_0565
section: 05  files:  wsj_0570
section: 05  files:  wsj_0529
section: 05  files:  wsj_0567
Attention!
section: 05  files:  wsj_0547
section: 05  files:  wsj_0585
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
section: 05  files:  wsj_0573
Attention!
Attention!
Attention!
section: 05  files:  wsj_0509
section: 05  files:  wsj_0502
section: 05  files:

Attention!
Attention!
Attention!
Attention!
Attention!
section: 06  files:  wsj_0690
section: 06  files:  wsj_0645
section: 06  files:  wsj_0671
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
section: 06  files:  wsj_0631
section: 06  files:  wsj_0692
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
Attention!
section: 06  files:  wsj_0634
Attention!
Attention!
Attention!
section: 06  files:  wsj_0674
section: 06  files:  wsj_0637
Attention!
section: 06  files:  wsj_0632
Attention!
Attention!
section: 06  files:  wsj_0650
section: 06  files:  wsj_0622
section: 06  files:  wsj_0672
section: 06  files:  wsj_0675
section: 06  files:  wsj_0612
section: 06  files:  wsj_0653
section: 06  files:  wsj_0626
section: 06  files:  wsj_0673
section: 06  files:  wsj_0664
Attention!
Attention!
Attention

In [ ]:
pdtb2.to_csv('pdtb2.csv')

In [8]:
check_number_of_errors()

error numbers for arg1  single: 30149  double:  1004  triple:  6169
error numbers for arg2  single: 29912  double:  1560  triple:  6556
